# Create Stores Content

In [1]:
import os, sys, random
import pandas as PD

In [2]:
BACKEND_PATH = os.path.dirname(os.path.dirname(os.getcwd()))
APPPROJ_PATH = os.path.join(BACKEND_PATH, 'app_proj')
sys.path.append(APPPROJ_PATH)
import notebooks as NT
print([x for x in dir(NT) if x[0]!='_' and len(x)>8])

['DataframeToDicts', 'GetRandom', 'NOTEBOOK_ENV']


In [3]:
import django
os.environ['DEPLOYMENT_ENV'] = NT.NOTEBOOK_ENV
os.environ['DJANGO_ALLOW_ASYNC_UNSAFE'] = 'True'
MODULE_PATH = os.path.dirname(os.getcwd())
os.chdir(os.path.dirname(MODULE_PATH))
django.setup()

env: DEV


In [4]:
import database as DB
print([x for x in dir(DB) if x[0]!='_' and len(x)>8])

['BaseManager', 'DeleteTable', 'GetNativeTableCounts', 'GetTableCounts', 'GetTableDictionary', 'InsertBulk', 'InsertSingle']


In [5]:
import django.db.models as JM
from django.contrib.auth import get_user_model

In [6]:
import emporium.models as EM
import engine.models as GM

import engine.logic.content as CT

In [7]:
%load_ext autoreload
%autoreload 2
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

### Get Guild

In [24]:
userMd = get_user_model().objects.filter(user_name='Admin')[0]
guildMd = GM.Guild.objects.GetOrNone(UserFK=userMd, Selected=True)
PD.DataFrame([guildMd.__dict__]).drop(['id', '_state'], axis=1, errors='ignore')

,UserFK_id,Name,LastPlayed,CreateDate,Selected,ThroneLevel,CampaignWorld,VaultGold,VaultStone,VaultGems,DungeonCheck
0,1,123,2024-09-05,2024-07-30,True,1,1,577,0,23,2024-09-05


In [11]:
# delete magic store for dev

storeLs = GM.MarketStore.objects.filter(GuildFK=guildMd).values()
PD.DataFrame(storeLs).drop(['id', '_state'], axis=1, errors='ignore')

,GuildFK_id,CreateDate,ThroneLevel,ResourceId,StoreType,RareProperties,Bought
0,21,2024-09-05,1,thief-agi-s1,common,"{'name': 'Heramis', 'agi': 3, 'cun': 0, 'mig':...",False
1,21,2024-09-05,1,thief-cun-s1,common,"{'name': 'Phenax', 'agi': 0, 'cun': 3, 'mig': ...",False
2,21,2024-09-05,1,thief-mig-s1,common,"{'name': 'Heptanus', 'agi': 0, 'cun': 0, 'mig'...",False
3,21,2024-09-05,1,feet-skl-l1-m0,rare,None,False
4,21,2024-09-05,1,hands-skl-l1-m0,rare,None,False
5,21,2024-09-05,1,feet-cmb-l1-m0,rare,None,False
6,21,2024-09-05,1,head-skl-l1-m0,rare,None,False


In [18]:
GM.MarketStore.objects.all().delete()

(7, {'engine.MarketStore': 7})

In [25]:
storeLs = GM.ThiefUnlocked.objects.filter(GuildFK=guildMd).values()
PD.DataFrame(storeLs).drop(['id', '_state'], axis=1, errors='ignore')

""


In [20]:
# unlocked blueprints report

unlockThief = EM.UnlockableThief.objects.filter(Stars__gt=1).values()

PD.DataFrame(unlockThief)#.drop(['id', '_state'], axis=1, errors='ignore')

,id,ResourceId,Class,Stars,UnlockThrone,StoreCost,StartTrait,RandomTraits
0,220,thief-agi-s2,Burglar,2,4,6700,agi 4,2
1,221,thief-cun-s2,Scoundrel,2,4,6700,cun 4,2
2,222,thief-mig-s2,Ruffian,2,4,6700,mig 4,2
3,223,thief-agi-s3,Burglar,3,7,11400,agi 5,4
4,224,thief-cun-s3,Scoundrel,3,7,11400,cun 5,4
5,225,thief-mig-s3,Ruffian,3,7,11400,mig 5,4
6,226,thief-agi-s4,Burglar,4,9,17100,agi 6,6
7,227,thief-cun-s4,Scoundrel,4,9,17100,cun 6,6
8,228,thief-mig-s4,Ruffian,4,9,17100,mig 6,6


In [ ]:
for th in unlockThief:
    th['IconCode'] = f"class-{th['Class'].lower()}-s{th['Stars']}"
    checkUnlock = GM.ThiefUnlocked.objects.GetOrNone(GuildFK=guildMd, ThiefFK__ResourceId=th['ResourceId'])
    th['Unlocked'] = True if checkUnlock else False
    
PD.DataFrame(unlockThief)